In [1]:
# !Rscript ../utils/install_r_libraries.r

In [2]:
%load_ext rpy2.ipython

In [3]:
import os
from datapaths import *

In [4]:
%%R
library(GenomicFeatures)
library(ChIPpeakAnno)
library(ChIPseeker)
library(clusterProfiler)
library(dplyr)
library(stringr)

R[write to console]: Loading required package: BiocGenerics

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: 
Attaching package: ‘S4Vectors’


R[write to console]: The following objects are masked from ‘package:base’:

    expand.grid, I, unname


R[write to console]: Lo

In [5]:
%%R -i F_GENCODE_STR
txdb_all_annotation <- makeTxDbFromGFF(F_GENCODE_STR)

R[write to console]: Import genomic features from the file as a GRanges object ... 
R[write to console]: OK

R[write to console]: Prepare the 'metadata' data frame ... 
R[write to console]: OK

R[write to console]: Make the TxDb object ... 
R[write to console]: OK



In [6]:
%%R
# https://www.biostars.org/p/140471/
gtf.file = F_GENCODE_STR
gtf.gr = rtracklayer::import(gtf.file) # creates a GRanges object
gtf.df = as.data.frame(gtf.gr)

genes = unique(gtf.df[ ,c("gene_id","gene_name")])
head(genes)

                gene_id     gene_name
1  ENSMUSG00000102693.1 4933401J01Rik
4  ENSMUSG00000064842.1       Gm26206
7  ENSMUSG00000051951.5          Xkr4
25 ENSMUSG00000102851.1       Gm18956
28 ENSMUSG00000103377.1       Gm37180
31 ENSMUSG00000104017.1       Gm37363


In [7]:
flipon_list = [
    str(D_FLIPONS_BED / 'actb_ssdna_enriched_g4.bed'),
    str(D_FLIPONS_BED / 'actb_ssdna_enriched_sidd.bed'),
    str(D_FLIPONS_BED / 'actb_ssdna_enriched_z-dna.bed'),
    str(D_FLIPONS_BED / 'actb_ssdna_enriched_h-dna.bed'),
]
flipon_list

['/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/actb_ssdna_enriched_g4.bed',
 '/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/actb_ssdna_enriched_sidd.bed',
 '/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/actb_ssdna_enriched_z-dna.bed',
 '/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/actb_ssdna_enriched_h-dna.bed']

In [9]:
%%R -i flipon_list

df_final <- data.frame()

for (flipon in flipon_list) {
  peakAnno <- annotatePeak(flipon, tssRegion=c(-3000, 3000), TxDb=txdb_all_annotation, verbose=FALSE)

  anno_df <- data.frame(peakAnno@anno)
  anno_df$group <- str_replace(str_split(flipon, 'enriched_')[[1]][2], '.bed', '')
  # res_df <- anno_df[grepl("Promoter", anno_df$annotation),][c('annotation', 'geneId')]
  # res_unique_genes_df <- distinct(res_df, geneId, .keep_all=TRUE)

  df_final <- bind_rows(df_final, anno_df)
  
  # print('--------------------------------------------')
  print(flipon)
  # print('--------------------------------------------')
  # print(peakAnno)
  # print(dim(anno_df))
}

df_final <- merge(df_final, genes, by.x='geneId', by.y='gene_id', all.x=TRUE)
# write.table(df_final, 'geneIds_by_group.tsv', append=FALSE, sep="\t", dec=".", row.names=FALSE, col.names = TRUE)
# df_final


[1] "/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/actb_ssdna_enriched_g4.bed"
[1] "/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/actb_ssdna_enriched_sidd.bed"
[1] "/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/actb_ssdna_enriched_z-dna.bed"
[1] "/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/actb_ssdna_enriched_h-dna.bed"


In [29]:
flipon_to_gene = %R df_final

flipon_to_gene = flipon_to_gene.sort_values(["group", "seqnames", "start"]).assign(
    annotation=lambda df: df["annotation"].apply(
        lambda x: x.split(" ")[0].strip() if "Intron" in x or "Exon" in x else x
    ),
    geneStrand=lambda df: df["geneStrand"].apply(lambda x: "+" if x == 1 else "-"),
)[["group", "V4", "gene_name", "geneStrand", "annotation"]]

flipon_to_gene.columns = [
    "Flipon",
    "Coordinates",
    "Gene Name",
    "Gene Strand",
    "Gene Feature",
]

flipon_to_gene


,Flipon,Coordinates,Gene Name,Gene Strand,Gene Feature
58993,g4,chr1:3014794-3014871,4933401J01Rik,+,Distal Intergenic
36086,g4,chr1:3099888-3099963,Gm26206,+,Promoter (2-3kb)
59200,g4,chr1:3287445-3287468,Gm18956,+,Intron
59339,g4,chr1:3472953-3472969,Gm37686,-,Intron
59505,g4,chr1:3535948-3535996,Gm7341,+,Intron
...,...,...,...,...,...
56627,z-dna,chrY:3866251-3866287,Gm8521,+,Distal Intergenic
57669,z-dna,chrY:4195990-4196008,Gm29038,-,Distal Intergenic
57047,z-dna,chrY:4202845-4202862,Gm28191,+,Distal Intergenic
57058,z-dna,chrY:4203100-4203146,Gm28191,+,Distal Intergenic


In [31]:
flipon_to_gene.to_csv(F_FLIPON_TO_GENE, sep='\t', index=False)